# 1. AUTOMATICALLY PARSING SOFTWARE REQUIREMENT TO TEST CASES

In [1]:
# import argparse

# parser = argparse.ArgumentParser(description='Arguments information.')

# parser.add_argument('-s', '--scenario-dir', dest='scenario_dir', type=str,
#                     help='A required scenario directory.', default='test-scenario')
# parser.add_argument('-st', '--stanford-tool-dir', dest='stanford_tool_dir', type=str,
#                     help='A required Stanford CoreNLP directory.', default='Stanford tools/stanford-corenlp-full-2017-06-09')
# parser.add_argument('-d', '--dictionary-path', dest='dictionary_path', type=str,
#                     help='A required dictionary file path. (JSON format', default='dictionary.json')
# parser.add_argument('-o', '--output-path', dest='output_path', type=str,
#                     help='A required output file path.', default='test_cases')
# parser.add_argument('-c', '--config-path', dest='config_path', type=str,
#                     help='A required configuration file path. (TXT format)', default='config.ini')

In [2]:
# args = parser.parse_args()
# scenario_dir = args.scenario_dir
# stanford_tool_dir = args.stanford_tool_dir
# dictionary_path = args.dictionary_path
# test_cases_txt_path = args.output_path + '.txt'
# test_cases_pkl_path = args.output_path + '.pkl'

In [3]:
scenario_dir = "test-scenario"
stanford_tool_dir = 'D:\\Project\\AI\\automation\\Automatic-Testing-Demo\\Automatic-Testing-Demo\\Stanford-tools\\stanford-corenlp-full-2017-06-09'
dictionary_path = 'dictionary.json'
test_cases_txt_path = 'test_cases.txt'
test_cases_pkl_path = 'test_cases.pkl'
config_path = 'config.ini'

In [4]:
import os
from stanfordcorenlp import StanfordCoreNLP
import json
from scenario import *

## 1.1 Reading test scenario from file and store it into structures

In [5]:
scenarios = []
# scenario_dir = "test-scenario"
for scenario in os.listdir(scenario_dir):
    with open(scenario_dir + "//" +  scenario, "r") as f:
        scenarios.append(f.read())

In [6]:
scenarios[0]

'User writes search box. EXPECTATION Data mining. User clicks search button. User clicks an item. EXPECTATION Data mining. User writes quantity. EXPECTATION 3. User clicks the button "Add to Cart".\n'

In [7]:
scenario = scenarios[0]

## 1.2 Calling Stanford CoreNLP and parsing test scenario

In [8]:
class Parser(object):
    def __init__(self, core_path, props):
        self.nlp = StanfordCoreNLP(core_path)
        self.props = props
    def annotate(self, text, props=None):
        if props is None:
            return json.loads(self.nlp.annotate(text, properties=self.props))
        return json.loads(self.nlp.annotate(text, properties=props))

In [9]:
props={'annotators': 'tokenize,ssplit,parse','pipelineLanguage':'en', 'outputFormat': 'json'}

In [10]:
parser = Parser(stanford_tool_dir, props)

In [11]:
parsed_scenario = parser.annotate(scenario)['sentences']

In [12]:
print "Parsing..."

Parsing...


In [13]:
processed_scenario = Scenario([]) # Constructor is still bad!

In [14]:
import re

In [15]:
# def construct_frame(parsed_sentence):
#     text = ""
#     # Check first token that is a condition or not
#     start_idx = 0
#     condition = ""
#     expectation = ""
#     has_expectation = False
#     if parsed_sentence['tokens'][0]['originalText'] in conditions:
#         start_idx = 1 
#         condition = parsed_sentence['tokens'][0]['originalText']
#     elif parsed_sentence['tokens'][0]['originalText'] in expectations:
#         start_idx = 1
#         has_expectation = True
#     # Process the rest of tokens
#     for idx in range(start_idx, len(parsed_sentence['tokens']) - 1):
#         token = parsed_sentence['tokens'][idx]
#         text += token['originalText'] + token['after']
#     # Check last token that is a punctuation or not
#     if parsed_sentence['tokens'][-1]['originalText'] != '.':
#         text += token['originalText']
#     if has_expectation == True:
#         processed_scenario.sentences[-1].expectation = text
#         return False ########################################## Notice when it no longer be function
#     # Create frames
#     frames = []
#     # Check if current sentence is active voice or not
#     is_active_voice = False
#     for dependency in parsed_sentence['enhancedPlusPlusDependencies']:
#         if dependency['dep'] == 'nsubj':
#             is_active_voice = True
#             frames.append(ActiveVoice_Frame(parsed_sentence))
#             break
#     if is_active_voice == False:
#         frames.append(Other_Frame(parsed_sentence))
#         print "Other!"
#     adding_sentence = Sentence(condition, text, frames, expectation)
#     # Append a Sentence instance to current scenario
#     get_mod_re = re.compile(r'\"([\w+ ]+)\"')# Get modifier in quotes
#     got_mod = get_mod_re.search(text)
#     if got_mod is not None:
#         adding_sentence.change_obj_mod(0, got_mod.group(1))
#     else:
#         if len(adding_sentence.frames[0].obj_mod) > 0:
#             s = ""
#             for obj_mod in adding_sentence.frames[0].obj_mod:
#                 s += obj_mod
#             adding_sentence.change_obj_mod(0, s)
#         else:
#             adding_sentence.change_obj_mod(0, adding_sentence.change_obj_mod(0, ''))
#     return adding_sentence

In [16]:
expectations_path = 'expectations.txt'
with open(expectations_path, 'r') as f:
    expectation_list = f.read()[:-1].split(';')
    if len(expectation_list) > 0:
        expectation_list = expectation_list
expectation_idx = 0

In [17]:
import ConfigParser
class Config_parser:
    def __init__(self, fileconfig):
        self.config = ConfigParser.ConfigParser();
        self.config.read(fileconfig);
    
    def get(self, session, key):
        return self.config.get(session, key);

In [18]:
config = Config_parser(config_path)

In [19]:
# Process scenario and create sentences and frames
conditions = ["IF", "THEN", "ELSE", "Y", "N"]
expectations = ["EXPECTATION"]
for sentence_idx in range(len(parsed_scenario)):
    parsed_sentence = parsed_scenario[sentence_idx]
    # Create text
#     adding_sentence = construct_frame(parsed_sentence)
#     if adding_sentence != False:
#         processed_scenario.add_sentence(adding_sentence)
    text = ""
    # Check first token that is a condition or not
    start_idx = 0
    condition = ""
    expectation = ""
    has_expectation = False
    if parsed_sentence['tokens'][0]['originalText'] in conditions:
        start_idx = 1 
        condition = parsed_sentence['tokens'][0]['originalText']
    elif parsed_sentence['tokens'][0]['originalText'] in expectations:
        start_idx = 1
        has_expectation = True
    # Process the rest of tokens
    for idx in range(start_idx, len(parsed_sentence['tokens']) - 1):
        token = parsed_sentence['tokens'][idx]
        text += token['originalText'] + token['after']
    # Check last token that is a punctuation or not
    if parsed_sentence['tokens'][-1]['originalText'] != '.':
        text += token['originalText']
    if has_expectation == True:
        if text == '__from_file':
            expec = expectation_list[expectation_idx]
#             processed_scenario.sentences[-1].expectation = expec
            expectation_idx += 1
            text = config.get('Variables', expec)
            processed_scenario.sentences[-1].expectation = text
        else:
            processed_scenario.sentences[-1].expectation = text
        
        continue
    # Create frames
    frames = []
    # Check if current sentence is active voice or not
    is_active_voice = False
    for dependency in parsed_sentence['enhancedPlusPlusDependencies']:
        if dependency['dep'] == 'nsubj':
            is_active_voice = True
            frames.append(ActiveVoice_Frame(parsed_sentence))
            break
    if is_active_voice == False:
        frames.append(Other_Frame(parsed_sentence))
        print "Non-ActiveVoiceSentence!"
    adding_sentence = Sentence(condition, text, frames, expectation)
    # Append a Sentence instance to current scenario
    get_mod_re = re.compile(r'\"([\w+ ]+)\"')# Get modifier in quotes
    got_mod = get_mod_re.search(text)
    if got_mod is not None:
        adding_sentence.change_obj_mod(0, got_mod.group(1))
    else:
        if len(adding_sentence.frames[0].obj_mod) > 0:
            s = ""
            for obj_mod in adding_sentence.frames[0].obj_mod:
                s += obj_mod
            adding_sentence.change_obj_mod(0, s)
        else:
            adding_sentence.change_obj_mod(0, adding_sentence.change_obj_mod(0, ''))
    processed_scenario.add_sentence(adding_sentence)

## 1.3 Contructing Graph for generating test cases

In [20]:
class Node(object):
    def __init__(self, sentence, children):
        self.sentence = sentence
        self.children = children
        self.parents = []

In [21]:
class Activity_Graph(object):
    def __init__(self, root):
        self.root = root
    
    def add_node(self, sentence_text, node):
        current_node = graph_dict[sentence_text]
        current_node.children.append(node)
        graph_dict[node.sentence.text] = node

In [22]:
root = Node(Sentence("", "ROOT", []), [])
# Graph dictionary that contains all nodes in activity graph helps us easier to iterate through all graph's nodes
graph_dict = {"ROOT": root}

In [23]:
activity_graph = Activity_Graph(root)

In [24]:
Y_count = N_count = 0
for idx in range(len(processed_scenario.sentences)):
    sentences = processed_scenario.sentences
#     print sentences[idx].text
    if idx == 0: # First node in graph
        activity_graph.add_node("ROOT", Node(sentences[idx], []))
    else: # If IF, THEN, ELSE are met
        if sentences[idx].condition == "THEN":
            Y_count += 1
        elif sentences[idx].condition == "ELSE":
            N_count += 1
        # Construct THEN, ELSE branches
        if sentences[idx].condition in ["THEN", "ELSE"]:
            sub_idx = idx - 1
            while sub_idx >= 0:
                if sentences[sub_idx].condition == "IF":
                    condition_node = graph_dict[sentences[sub_idx].text]
                    new_node = Node(sentences[idx], [])
                    condition_node.children.append(new_node)
                    graph_dict[new_node.sentence.text] = new_node
                    break
                sub_idx -= 1
        # Contruct ELSE branch if current condition is "N"
        elif sentences[idx].condition == "N":
            N_up = 1
            sub_idx = idx - 1
            while sub_idx >= 0:
                if sentences[sub_idx].condition == "ELSE":
                    if N_up == N_count:
                        condition_node = graph_dict[sentences[sub_idx].text]
                        new_node = Node(sentences[idx], [])
                        condition_node.children.append(new_node)
                        graph_dict[new_node.sentence.text] = new_node
                        N_count -= 1
                        break
                    else:
                        N_up += 1
                sub_idx -= 1
        # Construct THEN branch if current condition is "Y"
        elif sentences[idx].condition == "Y":
            Y_up = 1
            sub_idx = idx - 1
            while sub_idx >= 0:
                if sentences[sub_idx].condition == "THEN":
                    if Y_up == Y_count:
                        condition_node = graph_dict[sentences[sub_idx].text]
                        new_node = Node(sentences[idx], [])
                        condition_node.children.append(new_node)
                        graph_dict[new_node.sentence.text] = new_node
                        Y_count -= 1
                        break
                    else:
                        Y_up += 1
                sub_idx -= 1
        else: # Add node regularly when meeting no condition
            activity_graph.add_node(sentences[idx - 1].text, Node(sentences[idx], []))

In [25]:
redundant_sentences = []
idx = 0
while idx < len(graph_dict.keys()):
#     print "Lengths: ", len(graph_dict.keys())
    sentence = graph_dict.keys()[idx]
    node = graph_dict[sentence]
    parents = []

    if node.sentence.condition == "IF":
        for stn in graph_dict:
            nod = graph_dict[stn]
            idx_2 = 0
            while idx_2 < len(nod.children):
                if node == nod.children[idx_2]:
                    parents.append(nod)
                    del nod.children[idx_2]
                    idx_2 -= 1
                idx_2 +=1
                    
#             if node in nod.children:
#                 parents.append(nod)
#         print "Parents: ", parents
#         print "Children: ", node.children
        for parent in parents:
            parent.children += node.children
#         redundant_sentences.append(sentence)
#         print "Redundants: ", redundant_sentences
        graph_dict.pop(sentence, None)
        idx = 0
        continue
    idx += 1
# for sentence in redundant_sentences:
#     graph_dict.pop(sentence, None)

## 1.4 Dictionary for additional steps

In [26]:
# # Dictionary stores keywords commonly used in user stories and their associated test steps
# # Haven't used yet, I'll use it in future
# dictionary = {"quantity": ["User writes positive value.", "User writes negative value."],
              
#              }

In [27]:
with open(dictionary_path, 'r') as f:
    dic = f.read()
    dictionary = json.loads(dic)

In [28]:
additional_test_steps_text = []

In [29]:
# Create new branches by searching actors, objects, and actions from dictionary
# Search all nodes (sentences) in graph dictionary
for sentence in graph_dict:
    current_node = graph_dict[sentence]
    frames = current_node.sentence.frames
    # Iterate all frames of current sentence
    for frame in frames:
        # Iterate all object in current frame 
        for obj in frame.obj:
            # Check if current object is in the Dictionary or not
            if obj.lower() in dictionary:
                # Get all children of current node
                current_children = current_node.children
                # Iterate all texts in satistfied key
                new_children = []
                for text in dictionary[obj.lower()]:
                    # Create new children node and its children are the children of current node
                    new_children.append(Node(Sentence("", text, []), current_children))
                    # Append to additional test steps list for further processing
                    additional_test_steps_text.append(text)
                # Assign new children to children of current node
                graph_dict[sentence].children = new_children

In [30]:
additional_test_steps = {}

In [31]:
for test_step in additional_test_steps_text:
    parsed_test_step = parser.annotate(test_step)['sentences'][0]
    condition = "" # No cases for it now
    # Create frames
    frames = []
    # Check if current sentence is active voice or not
    is_active_voice = False
    for dependency in parsed_test_step['enhancedPlusPlusDependencies']:
        if dependency['dep'] == 'nsubj':
            is_active_voice = True
            frames.append(ActiveVoice_Frame(parsed_test_step))
            break
    if is_active_voice == False:
        frames.append(Other_Frame(parsed_test_step))
        print "Other!"
    adding_sentence = Sentence(condition, test_step, frames, expectation)
    # Append a Sentence instance to current scenario
    get_mod_re = re.compile(r'\"([\w+ ]+)\"')# Get modifier in quotes
    got_mod = get_mod_re.search(test_step)
    if got_mod is not None:
        adding_sentence.change_obj_mod(0, got_mod.group(1))
    else:
        if len(adding_sentence.frames[0].obj_mod) > 0:
            s = ""
            for obj_mod in adding_sentence.frames[0].obj_mod:
                s += obj_mod
            adding_sentence.change_obj_mod(0, s)
        else:
            adding_sentence.change_obj_mod(0, adding_sentence.change_obj_mod(0, ''))
    additional_test_steps[test_step] = adding_sentence

In [32]:
test_cases = []

In [33]:
# visited_node = [] # 
# path = []
# def printpath(path):
#     s = ''
#     for e in path:
#         s += str(e) + ' --> '
#     print s[:-5]
#     print ''

In [34]:
# def DFS(root):
#     if len(root.children) == 0: # leaf node
#         test_case = []
#         for p in path:
#             if p in graph_dict:
#                 test_case.append(graph_dict[p].sentence)
#             elif p in additional_test_steps:
#                 test_case.append(additional_test_steps[p])
#         test_cases.append(test_case)
#         printpath(path)
#         return
#     for child in root.children:
# #         if child.sentence.text not in visited_node:
#         path.append(child.sentence.text)
#         DFS(child)
#         path.pop();

In [35]:
visited_node = [] # 
path = []
def printpath(path):
    s = ''
    for e in path:
        if len(e.expectation) > 0:
            s += str(e.text) + '. EXPECTATION ' + e.expectation + ' --> '
        else:
            s += str(e.text) + ' --> '
    print s[:-5]
    print ''

In [36]:
def DFS(root):
    if len(root.children) == 0: # leaf node
        test_case = []
        for p in path:
            if p.text in graph_dict:
                test_case.append(graph_dict[p.text].sentence)
            elif p.text in additional_test_steps:
                test_case.append(additional_test_steps[p.text])
        test_cases.append(test_case)
        printpath(path)
        return
    for child in root.children:
#         if child.sentence.text not in visited_node:
        path.append(child.sentence)
        DFS(child)
        path.pop();

In [37]:
DFS(root)

User writes search box. EXPECTATION Data mining --> User clicks search button --> User clicks an item. EXPECTATION Data mining --> User writes quantity. EXPECTATION 3 --> User writes positive value. --> User clicks the button "Add to Cart"

User writes search box. EXPECTATION Data mining --> User clicks search button --> User clicks an item. EXPECTATION Data mining --> User writes quantity. EXPECTATION 3 --> User writes negative value. --> User clicks the button "Add to Cart"



## 1.5 Writing generated test cases to file

In [38]:
# for test_case in test_cases:
#     for sentence in test_case:
#         print "Sentence: ", sentence.text
#         print "Action: ", sentence.frames[0].action[0]
#         print "Obj: ", sentence.frames[0].obj[0]
#         print "Obj_mod: ", sentence.frames[0].obj_mod

In [39]:
# Print test scenario to file
s = ''
for test_case in test_cases:
    for sentence in test_case:
        s += sentence.text + u' --> ';
    s = s[:-5]
    s += '\n'
    
with open(test_cases_txt_path, 'w') as f:
    f.write(s)

In [40]:
import pickle

In [41]:
with open(test_cases_pkl_path, 'wb') as f:
    pickle.dump(test_cases, f, pickle.HIGHEST_PROTOCOL)